# Correcting for our Amplifier's non-linear control

As discussed in the main notebook, we want to control our [variable gain amplifier (VGA)](https://www.nooelec.com/store/downloads/dl/file/id/103/product/334/vega_datasheet_revision_1.pdf) linearly for our AGC.
The main complication here is that our control signal to the VGA, $V_c$, is not linear with respect to gain! We will need to compensate for this when generating our control output.

The [Nooelec datasheet](https://www.nooelec.com/store/downloads/dl/file/id/103/product/334/vega_datasheet_revision_1.pdf) for the VeGA amplifier shows the following relationship between the control voltage and output gain (in decibels).

![control voltage to gain plot for Nooelec VeGA](./assets/vega_response.svg)

Note that the middle section of the graph is approximately linear in dB, which is an indication this compensation could be performed in the logarithmic domain. One option is to implement a logarithm in the FPGA, much like we did in the [baseband AGC notebook](1_Baseband_AGC). However, since we only have a 6-bit control for this amplifier, we will simply make a look-up table to map our digital control to appropriate $V_c$ values.

## Modelling the VeGA's response

We've very roughly recreated the response from the datasheet for a $1000$ MHz signal via visual inspection (read as: "We used GIMP").
Let's plot this below, now against a linear y-axis.

In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

In [ ]:
gain_db = [13,13,14,15,17,20,23,26,30,34,37,38,40,40,40.5,40.5,40.5]
gain_lin = [10**(db/20) for db in gain_db]
vc      = [i*0.2 for i in range(len(gain_db))]

frame = pd.DataFrame({'Vc': vc, 'Gain (dB)':gain_db, 'Gain (lin)':gain_lin})

px.line(frame, 'Vc', 'Gain (lin)')

The "S" shape of this response is characteristic of a sigmoid function. We will need the inverse of this response in order to compensate for it, so let's try modelling it mathematically with the help of `scipy`. First, let's normalise the graph above --- we don't really care about the absolute values of decibel gain or $V_c$, just their relationship.

In [ ]:
normalise = lambda xs : [(x-min(xs))/(max(xs)-min(xs)) for x in xs]
gain_lin = normalise(gain_lin)
vc       = normalise(vc)
frame    = pd.DataFrame({'Vc': vc, 'Gain (lin)':gain_lin})

px.line(frame, 'Vc', 'Gain (lin)')

We know we're expecting some sort of sigmoid function, so let's define such a function  (and its inverse) with parameters for scaling and offsets in both axes. Since the y-axis is between 0 and 1, we know the y scaling factor and y offset ahead of time --- let's fix those values now too.

In [ ]:
np.seterr(divide='ignore')

def sigmoid(xs, x_scaling, y_scaling, x_offset, y_offset):
    return y_scaling * np.tanh(xs * x_scaling - x_offset) + y_offset

def inv_sigmoid(xs, x_scaling, y_scaling, x_offset, y_offset):
    y = (1/x_scaling) * (np.arctanh((xs - y_offset) / y_scaling) + x_offset)
    y[y == -np.inf] = 0
    y[y == np.inf] = 1
    return y

our_sigmoid     = lambda x, x_scaling, x_offset :     sigmoid(x, x_scaling, 0.5, x_offset, 0.5)
our_inv_sigmoid = lambda x, x_scaling, x_offset : inv_sigmoid(x, x_scaling, 0.5, x_offset, 0.5)

Given our known $V_c$ values and this sigmoid function, we can ask `scipy` to find the sigmoid function parameters that best represent our gain control. 

In [ ]:
popt, pcov = curve_fit(our_sigmoid, np.array(vc), gain_lin, bounds=(0, [100, 100]))
frame['Gain Model (lin)'] = our_sigmoid(np.array(vc), popt[0], popt[1])


frame_time = pd.melt(frame, id_vars=['Vc'], value_vars=['Gain (lin)', 'Gain Model (lin)'], var_name='c', value_name='y')
px.line(frame_time, x='Vc', y='y', color='c')

As we can see from the visualisation, this model does seem to closely resemble our gain/$V_c$ response.

## Generating a calibration lookup table

Now that we have modelled this mathematically, we can implement this compensation with a simple lookup table of values using the inverse sigmoid function.Let's generate the contents of the lookup table ROM now. Our output is 6 bits, so we can fit our LUT into one BRAM when configured as 4Kx9 (i.e. a 12 bit address). Note that we're using more than 6 bits on the input because of the logarithmic nature of the output --- using 6 bits at the input would result in many of the possible output words being unoccupied. In maths speak, it'd be non-injective and non-surjective. In practice it appears that our 12 bit address is enough to at least make this a surjective function.

In [ ]:
addr_bits = 12
word_bits = 6
quantised_gain = normalise(range(2**addr_bits))
xs = our_inv_sigmoid(np.array(quantised_gain), popt[0], popt[1])

# Constrain to normal range
xs = [min(1, max(0,x)) for x in xs]

quantised_vc = normalise(xs)
scaled_quantised_vc = [int((2**word_bits-1) * vc) for vc in quantised_vc]

np.array(scaled_quantised_vc)

In [ ]:
print("""
lutData :: Vec 4096 (Unsigned 6)
lutData = map unpack $""")

print('  ', end='')
for (i,b) in enumerate(scaled_quantised_vc):
    print('0b{0:06b} :> '.format(int(b)), end='')
    if i % 8 == 7: 
        print("\n  ", end='')

We can confirm that we're using the full scale of the 6 output bits, and we're good to copy and paste this into our HDL source!

### References

1: https://www.nooelec.com/store/downloads/dl/file/id/103/product/334/vega_datasheet_revision_1.pdf "VeGA Datasheet"

2: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html "Docs for scipy curve fitting"

3: https://www.xilinx.com/support/documentation/user_guides/ug573-ultrascale-memory-resources.pdf "UG573 UltraScale Architecture
Memory Resources"